preprocess the binance data to only consist of ohlcv features and save it to a csv file

we also had to convert the timestamp as it was mixed up with ms and s

source: https://data.binance.vision/?prefix=data/spot/monthly/klines/BTCUSDT/1m/

In [29]:
import os
import pandas as pd
from datetime import datetime


In [30]:
# Directory path
dir_path = "./torchtrade/data/binance_btc_1m_raw/"
columns = ["open time", "open", "high", "low", "close", "volume", "close time", "quote asset volume", "number of trades", "taker buy base asset volume", "taker buy quote asset volume", "ignore"]

In [31]:

# Get list of relevant CSV files
files = [f for f in os.listdir(dir_path) if f.startswith("BTCUSDT-1m-") and f.endswith(".csv")]

# Extract dates and pair with filenames
file_dates = []
for f in files:
    date_str = f.split("-1m-")[1].replace(".csv", "")
    date = datetime.strptime(date_str, "%Y-%m")
    file_dates.append((f, date))

# Sort by date in ascending order (historical order)
sorted_files = sorted(file_dates, key=lambda x: x[1])

# Load and concatenate DataFrames
dfs = []
for f, _ in sorted_files:
    df = pd.read_csv(os.path.join(dir_path, f), header=None, usecols=[0, 1, 2, 3, 4, 5]) # only open, high, low, close, volume
    dfs.append(df)

# Concatenate all DataFrames
full_df = pd.concat(dfs, ignore_index=True)
# adapt time stamp error

#Function to convert timestamps
def convert_timestamp(value):
    try:
        value = float(value)
        # Determine unit by number of digits/magnitude
        if value > 1e15:  # 16+ digits → microseconds
            return pd.to_datetime(value, unit='us')
        elif value > 1e12:  # 13-15 digits → milliseconds
            return pd.to_datetime(value, unit='ms')
        elif value > 1e9:  # 10-12 digits → seconds
            return pd.to_datetime(value, unit='s')
        else:
            return pd.NaT
    except (ValueError, TypeError):
        return pd.NaT

full_df[0] = full_df[0].apply(convert_timestamp)

#full_df[0] = pd.to_datetime(full_df[0], unit='ms', errors='coerce')



In [41]:
df = full_df.iloc[1694835:]

In [44]:
df

,0,1,2,3,4,5
1694835,2023-03-24 14:00:00,28079.99,28079.99,27901.06,27925.59,293.30587
1694836,2023-03-24 14:01:00,27925.60,27925.60,27850.00,27850.00,237.33799
1694837,2023-03-24 14:02:00,27850.01,27870.28,27835.00,27867.87,393.15752
1694838,2023-03-24 14:03:00,27866.96,27945.71,27857.58,27862.49,199.48266
1694839,2023-03-24 14:04:00,27862.50,27889.50,27857.57,27858.24,86.33641
...,...,...,...,...,...,...
3109510,2025-11-30 23:55:00,90395.39,90446.94,90394.31,90446.93,7.42720
3109511,2025-11-30 23:56:00,90446.93,90477.03,90404.25,90404.26,7.19218
3109512,2025-11-30 23:57:00,90404.26,90449.92,90380.00,90415.79,7.05179
3109513,2025-11-30 23:58:00,90415.78,90415.79,90339.70,90357.66,101.81945


In [20]:
full_df.to_csv("./btcusdt_spot_1m_01_2020_to_12_2025.csv", index=False)

In [11]:
import datasets
import pandas as pd


In [23]:
dataset = datasets.Dataset.from_csv("./torchtrade/data/btcusdt_1min.csv")

In [24]:
dataset = dataset.rename_columns({"time": "0", "open":"1", "high": "2", "low": "3", "close": "4"})

In [25]:
dataset = datasets.DatasetDict({"train": dataset})

In [28]:
dataset["train"].to_pandas()

,0,1,2,3,4
0,2022-06-13T03:00:00+03:00,26574.53,26684.86,26531.55,26662.00
1,2022-06-13T03:01:00+03:00,26662.01,26770.28,26598.97,26672.04
2,2022-06-13T03:02:00+03:00,26672.05,26685.10,26610.00,26628.76
3,2022-06-13T03:03:00+03:00,26628.75,26635.26,26500.00,26598.66
4,2022-06-13T03:04:00+03:00,26598.66,26638.86,26461.66,26483.40
...,...,...,...,...,...
23955,2022-06-29T18:15:00+03:00,20069.12,20082.08,20044.42,20052.36
23956,2022-06-29T18:16:00+03:00,20052.35,20068.83,20050.69,20068.82
23957,2022-06-29T18:17:00+03:00,20068.82,20069.14,20055.56,20058.06
23958,2022-06-29T18:18:00+03:00,20058.06,20058.06,20044.19,20044.19


In [35]:
dataset.push_to_hub("Torch-Trade/btcusdt_spot_1m_01_2020_to_12_2025")

Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00,  2.91ba/s]
Processing Files (0 / 1): 100%|█████████▉|  101MB /  101MB, 10.1MB/s  
New Data Upload: 100%|██████████|  101MB /  101MB, 10.1MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:12<00:00, 12.68s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Torch-Trade/btcusdt_spot_1m_01_2020_to_12_2025/commit/aba9e986f3545a6f3434b09e3220f7176455661a', commit_message='Upload dataset', commit_description='', oid='aba9e986f3545a6f3434b09e3220f7176455661a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Torch-Trade/btcusdt_spot_1m_01_2020_to_12_2025', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Torch-Trade/btcusdt_spot_1m_01_2020_to_12_2025'), pr_revision=None, pr_num=None)

In [3]:
compare = datasets.load_dataset("Torch-Trade/btcusdt_spot_1m_01_2020_to_12_2025")["train"]

In [ ]:
df = compare.to_pandas()


AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [47]:
df

,0,1,2,3,4,5
1694835,2023-03-24 14:00:00,28079.99,28079.99,27901.06,27925.59,293.30587
1694836,2023-03-24 14:01:00,27925.60,27925.60,27850.00,27850.00,237.33799
1694837,2023-03-24 14:02:00,27850.01,27870.28,27835.00,27867.87,393.15752
1694838,2023-03-24 14:03:00,27866.96,27945.71,27857.58,27862.49,199.48266
1694839,2023-03-24 14:04:00,27862.50,27889.50,27857.57,27858.24,86.33641
...,...,...,...,...,...,...
3109510,2025-11-30 23:55:00,90395.39,90446.94,90394.31,90446.93,7.42720
3109511,2025-11-30 23:56:00,90446.93,90477.03,90404.25,90404.26,7.19218
3109512,2025-11-30 23:57:00,90404.26,90449.92,90380.00,90415.79,7.05179
3109513,2025-11-30 23:58:00,90415.78,90415.79,90339.70,90357.66,101.81945


In [48]:
df = df.rename(columns={'0': 'timestamp', '1': 'open', '2': 'high', '3': 'low', '4': 'close', '5': 'volume'})

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Convert price/volume columns to numeric (if not already)
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric, errors='coerce')

# Sort by timestamp
df = df.sort_values('timestamp').reset_index(drop=True)

# Compute time differences in minutes
df['delta_min'] = df['timestamp'].diff().dt.total_seconds() / 60

# Find gaps where delta > 1 (ignoring the first NaN)
gaps = df[df['delta_min'] > 1].copy()
if not gaps.empty:
    gaps['previous_timestamp'] = gaps['timestamp'].shift(1)
    gaps['gap_start'] = gaps['previous_timestamp']
    gaps['gap_end'] = gaps['timestamp']
    gaps['gap_size_min'] = gaps['delta_min'] - 1
    print("Gaps found:")
    print(gaps[['gap_start', 'gap_end', 'gap_size_min']])
else:
    print("No gaps found - all timestamps are consecutive by 1 minute.")

# Overall summary
start_ts = df['timestamp'].min()
end_ts = df['timestamp'].max()
expected_rows = int((end_ts - start_ts).total_seconds() / 60) + 1
actual_rows = len(df)
missing_count = expected_rows - actual_rows
print(f"Start: {start_ts}")
print(f"End: {end_ts}")
print(f"Actual rows: {actual_rows}")
print(f"Expected rows for consecutive 1-min intervals: {expected_rows}")
print(f"Total missing entries: {missing_count}")

KeyError: 'timestamp'

In [48]:
train_df = df[df[0] < "2025-01-01"]
test_df  = df[df[0] >= "2025-01-01"]

KeyError: 0

In [50]:
train_df = df[df['0'] < "2025-01-01"]
test_df  = df[df['0'] >= "2025-01-01"]

In [1]:
train_df

NameError: name 'train_df' is not defined